# BigData Environment - Exemplo de Integração

Este notebook demonstra como usar os serviços integrados do ambiente BigData:
- **Spark**: Processamento de dados distribuído
- **MinIO**: Storage de objetos
- **Airflow**: Orquestração de workflows
- **Jenkins**: CI/CD

## 1. Configuração Inicial

In [ ]:
# Instalar pacotes necessários se não estiverem disponíveis
import subprocess
import sys

required_packages = ['minio', 'boto3']
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max, min
import matplotlib.pyplot as plt
import seaborn as sns
from minio import Minio
import io
import json

# Configurar visualização
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print("Bibliotecas importadas com sucesso!")

## 🔧 Verificação do Ambiente

**Se você encontrar o erro `ModuleNotFoundError: No module named 'pyspark'`, execute a célula abaixo:**

In [ ]:
# 🔧 Verificação e Instalação do PySpark (se necessário)
try:
    import pyspark
    print(f"✅ PySpark já está instalado! Versão: {pyspark.__version__}")
except ImportError:
    print("⚠️  PySpark não encontrado. Instalando...")
    import subprocess
    import sys
    
    # Instalar PySpark
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyspark==3.5.0"])
    
    # Tentar importar novamente
    import pyspark
    print(f"✅ PySpark instalado com sucesso! Versão: {pyspark.__version__}")

# Testar importações básicas
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, count, avg, max, min
    print("✅ Módulos do PySpark importados com sucesso!")
except Exception as e:
    print(f"❌ Erro ao importar módulos do PySpark: {e}")
    print("💡 Reinicie o kernel e tente novamente.")

## 2. Conexão com Spark

In [ ]:
# Criar sessão Spark conectada ao cluster
spark = SparkSession.builder \
    .appName("Jupyter-Spark-Integration") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print(f"Spark Session criada: {spark.sparkContext.appName}")
print(f"Spark UI: http://localhost:8081")
print(f"Versão do Spark: {spark.version}")

## 3. Conexão com MinIO

In [ ]:
# Configurar cliente MinIO
minio_client = Minio(
    "minio:9000",
    access_key="minioadmin",
    secret_key="minioadmin123",
    secure=False
)

# Listar buckets disponíveis
try:
    buckets = minio_client.list_buckets()
    print("Buckets disponíveis no MinIO:")
    for bucket in buckets:
        print(f"  - {bucket.name} (criado em: {bucket.creation_date})")
except Exception as e:
    print(f"Erro ao conectar com MinIO: {e}")

## 4. Criação e Processamento de Dados

In [ ]:
# Criar dataset de exemplo
np.random.seed(42)
n_samples = 1000

data = {
    'id': range(1, n_samples + 1),
    'nome': [f'Usuario_{i}' for i in range(1, n_samples + 1)],
    'idade': np.random.randint(18, 70, n_samples),
    'departamento': np.random.choice(['TI', 'Vendas', 'RH', 'Marketing', 'Financeiro'], n_samples),
    'salario': np.random.normal(5000, 1500, n_samples).round(2),
    'experiencia': np.random.randint(0, 20, n_samples),
    'satisfacao': np.random.uniform(1, 5, n_samples).round(1)
}

# Criar DataFrame Pandas
df_pandas = pd.DataFrame(data)
print("Dataset criado com sucesso!")
print(f"Shape: {df_pandas.shape}")
df_pandas.head()

In [ ]:
# Converter para Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

print("Esquema do DataFrame Spark:")
df_spark.printSchema()

print("\nPrimeiras 10 linhas:")
df_spark.show(10)

## 5. Análise de Dados com Spark

In [ ]:
# Estatísticas básicas
print("=== Estatísticas Básicas ===")
df_spark.describe().show()

# Análise por departamento
print("\n=== Análise por Departamento ===")
dept_stats = df_spark.groupBy("departamento") \
    .agg(
        count("*").alias("total_funcionarios"),
        avg("idade").alias("idade_media"),
        avg("salario").alias("salario_medio"),
        avg("experiencia").alias("experiencia_media"),
        avg("satisfacao").alias("satisfacao_media")
    ) \
    .orderBy("salario_medio", ascending=False)

dept_stats.show()

## 6. Salvando Dados no MinIO

In [ ]:
# Salvar dados originais no MinIO via Spark
try:
    print("Salvando dados no MinIO...")
    
    # Salvar dados originais
    df_spark.write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv("s3a://jupyter-data/datasets/funcionarios")
    
    # Salvar estatísticas por departamento
    dept_stats.write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv("s3a://jupyter-data/datasets/estatisticas_departamento")
    
    print("Dados salvos com sucesso no MinIO!")
    
except Exception as e:
    print(f"Erro ao salvar no MinIO via Spark: {e}")
    print("Tentando salvar via cliente MinIO...")
    
    # Fallback: salvar via cliente MinIO
    csv_buffer = io.StringIO()
    df_pandas.to_csv(csv_buffer, index=False)
    csv_data = csv_buffer.getvalue().encode('utf-8')
    
    minio_client.put_object(
        "jupyter-data",
        "datasets/funcionarios.csv",
        io.BytesIO(csv_data),
        len(csv_data),
        content_type="text/csv"
    )
    print("Dados salvos via cliente MinIO!")

## 7. Visualizações

In [ ]:
# Converter estatísticas para Pandas para visualização
dept_stats_pandas = dept_stats.toPandas()

# Criar visualizações
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análise de Funcionários por Departamento', fontsize=16)

# Gráfico 1: Número de funcionários por departamento
axes[0, 0].bar(dept_stats_pandas['departamento'], dept_stats_pandas['total_funcionarios'])
axes[0, 0].set_title('Funcionários por Departamento')
axes[0, 0].set_xlabel('Departamento')
axes[0, 0].set_ylabel('Número de Funcionários')
axes[0, 0].tick_params(axis='x', rotation=45)

# Gráfico 2: Salário médio por departamento
axes[0, 1].bar(dept_stats_pandas['departamento'], dept_stats_pandas['salario_medio'])
axes[0, 1].set_title('Salário Médio por Departamento')
axes[0, 1].set_xlabel('Departamento')
axes[0, 1].set_ylabel('Salário Médio (R$)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Gráfico 3: Idade média por departamento
axes[1, 0].bar(dept_stats_pandas['departamento'], dept_stats_pandas['idade_media'])
axes[1, 0].set_title('Idade Média por Departamento')
axes[1, 0].set_xlabel('Departamento')
axes[1, 0].set_ylabel('Idade Média (anos)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Gráfico 4: Satisfação média por departamento
axes[1, 1].bar(dept_stats_pandas['departamento'], dept_stats_pandas['satisfacao_media'])
axes[1, 1].set_title('Satisfação Média por Departamento')
axes[1, 1].set_xlabel('Departamento')
axes[1, 1].set_ylabel('Satisfação Média (1-5)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Análise de Correlação

In [ ]:
# Análise de correlação
numeric_columns = ['idade', 'salario', 'experiencia', 'satisfacao']
correlation_matrix = df_pandas[numeric_columns].corr()

# Heatmap de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Matriz de Correlação')
plt.show()

# Scatter plot: Experiência vs Salário
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pandas['experiencia'], df_pandas['salario'], 
                     c=df_pandas['satisfacao'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Satisfação')
plt.xlabel('Experiência (anos)')
plt.ylabel('Salário (R$)')
plt.title('Relação entre Experiência, Salário e Satisfação')
plt.show()

## 9. Limpeza e Finalização

In [ ]:
# Finalizar sessão Spark
spark.stop()
print("Sessão Spark finalizada.")

print("\n=== Resumo da Execução ===")
print(f"✅ Dados processados: {len(df_pandas)} registros")
print(f"✅ Departamentos analisados: {df_pandas['departamento'].nunique()}")
print(f"✅ Visualizações criadas: 6 gráficos")
print(f"✅ Dados salvos no MinIO")
print(f"\n🔗 Acesse os outros serviços:")
print(f"   • Airflow: http://localhost:8080 (admin/admin)")
print(f"   • Spark UI: http://localhost:8081")
print(f"   • MinIO: http://localhost:9001 (minioadmin/minioadmin123)")
print(f"   • Jenkins: http://localhost:8082 (admin/admin)")
print(f"   • Flower (Celery): http://localhost:5555")